# Azure AI Agents with Model Context Protocol (MCP) 支援 - Python

此筆記本展示如何在 Python 中使用 Azure AI Agents 與 Model Context Protocol (MCP) 工具。它展示如何建立一個智能代理，透過無需密鑰的身份驗證，利用外部 MCP 伺服器（例如 Microsoft Learn）來增強功能。


## 安裝所需的 Python 套件

首先，我們需要安裝必要的 Python 套件：
- **azure-ai-projects**：核心 Azure AI Projects SDK
- **azure-ai-agents**：Azure AI Agents SDK，用於建立和管理代理
- **azure-identity**：使用 DefaultAzureCredential 提供無密鑰身份驗證
- **mcp**：Python 的 Model Context Protocol 實現


## 無密鑰身份驗證的好處

此筆記本展示了**無密鑰身份驗證**，提供以下幾個優勢：
- ✅ **無需管理 API 密鑰** - 使用基於 Azure 身份的身份驗證
- ✅ **增強安全性** - 無需在代碼或配置文件中存儲機密
- ✅ **自動憑證輪換** - Azure 負責憑證生命周期管理
- ✅ **基於角色的訪問控制** - 使用 Azure RBAC 提供精細化的權限管理
- ✅ **多環境支持** - 在開發和生產環境中無縫運作

`DefaultAzureCredential` 會自動選擇最佳可用的憑證來源：
1. **托管身份**（在 Azure 中運行時）
2. **Azure CLI** 憑證（本地開發期間）
3. **Visual Studio** 憑證
4. **環境變數**（如果已配置）
5. **互動式瀏覽器**身份驗證（作為備選方案）


## 無密鑰身份驗證設置

**無密鑰身份驗證的先決條件：**

### 用於本地開發：
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### 用於 Azure 環境：
- 在您的 Azure 資源上啟用 **系統分配的托管身份**
- 為托管身份分配適當的 **RBAC 角色**：
  - `Cognitive Services OpenAI User` 用於訪問 Azure OpenAI
  - `AI Developer` 用於訪問 Azure AI 項目

### 環境變數（可選）：
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**不需要 API 密鑰或連接字符串！** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## 匯入所需的庫

匯入必要的 Python 模組：
- **os, time**：標準 Python 庫，用於環境變數和延遲
- **AIProjectClient**：Azure AI Projects 的主要客戶端
- **DefaultAzureCredential**：Azure 服務的無密鑰身份驗證
- **與 MCP 相關的類別**：用於建立和管理 MCP 工具以及處理審批


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## 配置 MCP 伺服器設定

使用環境變數設置 MCP 伺服器配置，並提供預設值作為後備選項：
- **MCP_SERVER_URL**：MCP 伺服器的 URL（預設為 Microsoft Learn API）
- **MCP_SERVER_LABEL**：用於識別 MCP 伺服器的標籤（預設為 "mslearn"）

此方法可讓不同環境之間的配置更加靈活。


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## 建立 Azure AI Project Client（無密鑰認證）

使用**無密鑰認證**初始化 Azure AI Project 客戶端：
- **endpoint**：Azure AI Foundry 項目端點 URL
- **credential**：使用 `DefaultAzureCredential()` 進行安全的無密鑰認證
- **無需 API 密鑰**：自動發現並使用最佳可用的憑證

**認證流程：**
1. 檢查是否有受管理的身份（在 Azure 環境中）
2. 回退至 Azure CLI 憑證（用於本地開發）
3. 根據需要使用其他可用的憑證來源

此方法消除了在代碼中管理 API 密鑰或連接字串的需求。


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## 建立 MCP 工具定義

建立一個 MCP 工具，連接到 Microsoft Learn MCP 伺服器：
- **server_label**：MCP 伺服器的識別名稱
- **server_url**：MCP 伺服器的 URL 端點
- **allowed_tools**：可選的清單，用於限制可以使用的工具（空清單允許所有工具）

此工具將使代理能夠訪問 Microsoft Learn 的文件和資源。


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## 建立代理並執行對話（無密鑰工作流程）

這部分詳細說明了完整的**無密鑰代理工作流程**：

1. **建立 AI 代理**：使用 GPT-4.1 nano 模型和 MCP 工具設置代理
2. **建立對話線程**：建立一個用於溝通的對話線程
3. **發送訊息**：詢問代理有關 Azure OpenAI 與 OpenAI 的差異
4. **處理工具批准**：在需要時自動批准 MCP 工具調用
5. **監控執行**：追蹤代理的進度並處理任何必要的操作
6. **顯示結果**：展示對話內容和工具使用詳情

**無密鑰功能：**
- ✅ **無硬編碼密鑰** - 所有身份驗證均由 Azure 身份處理
- ✅ **默認安全** - 使用基於角色的訪問控制
- ✅ **簡化部署** - 無需憑證管理
- ✅ **審計友好** - 所有訪問均通過 Azure 身份進行追蹤

代理將使用 MCP 工具安全地訪問 Microsoft Learn 資源，無需管理 API 密鑰。


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
